# Cardio Analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from models.mlp import BlackBoxModel
from models.lr import LogisticRegression
from models.svm import LinearSVM
from models.rbf import RBFNet

pd.set_option('display.max_columns', None)

%reload_ext autoreload
%autoreload 2

## Read and Process Data

In [ ]:
df_ = pd.read_csv('data/cardio/cardio.csv', sep=';')
df = df_.drop(columns=['id'], axis=1).copy()

In [ ]:
target_name = 'cardio'
target = df[target_name].replace({})

In [ ]:
df.head()

## Model Training

In [ ]:
features = [
    'age', 
    'gender', 
    'height', 
    'weight', 
    'ap_hi', 
    'ap_lo', 
    'cholesterol',
    'gluc', 
    'smoke', 
    'alco', 
    'active'
]

df_X = df[features].copy()
df_y = df[target_name].copy()

In [ ]:
seed = 42

np.random.seed(seed)  # for reproducibility


# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=seed)

std = X_train.std()
mean = X_train.mean()

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# X_train, X_test, y_train, y_test = X_train.values, X_test.values, y_train.values, y_test.values

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)

# Initialize the model, loss function, and optimizer
# model = LinearSVM(input_dim=X_train.shape[1])
# model = LogisticRegression(input_dim=X_train.shape[1])
model = BlackBoxModel(input_dim=X_train.shape[1])
# model = RBFNet(input_dim=X_train.shape[1], hidden_dim=X_train.shape[1], output_dim=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 300
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)

    # Convert outputs to binary using 0.5 as threshold
    y_pred_tensor = (test_outputs > 0.5).float()
    correct_predictions = (y_pred_tensor == y_test_tensor).float().sum()
    accuracy = correct_predictions / y_test_tensor.shape[0]

accuracy.item()

## Counterfactual Explanation

In [ ]:
sample_num = 100
delta=0.15
alpha=0.1
N=10
explain_columns = [
    'age', 
    'gender', 
    'height', 
    'weight', 
    'ap_hi', 
    'ap_lo', 
    'cholesterol',
    'gluc', 
    'smoke', 
    'alco', 
    'active'
]

indice = (X_test.sample(sample_num)).index

df_explain = X_test.loc[indice]

# X = X_test.loc[indice].values
y = model(torch.FloatTensor(df_explain.values))

y_target = torch.round(torch.distributions.beta.Beta(0.2, 0.3).sample((sample_num,)))

y_true = y_test.loc[indice]

# Sort the arrays
y_target_sorted = np.sort(y_target.numpy())
y_true_sorted = np.sort(y_true)
y_sorted = np.sort(y.squeeze().detach().numpy())

# Generate quantiles
quantiles = np.linspace(0, 1, sample_num)

# Plotting
plt.figure(figsize=(6, 5))
plt.plot(quantiles, y_target_sorted, label='y_target (Beta Dist.)', color='red')
plt.plot(quantiles, y_true_sorted, label='y_true', color='black')
plt.plot(quantiles, y_sorted, label='y_model', color='green')


plt.title('Quantile Function')
plt.xlabel('Quantiles')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from explainers.dce import DistributionalCounterfactualExplainer

explainer = DistributionalCounterfactualExplainer(
    model=model, 
    df_X=df_explain, 
    explain_columns=explain_columns,
    y_target=y_target, 
    lr=1e-1, 
    n_proj=10,
    delta=delta)

In [ ]:
np.sqrt(explainer.wd.distance(y, y_target, delta=delta)[0].item())

In [ ]:
explainer.wd.distance_interval(y, y_target, delta=delta, alpha=alpha, bootstrap=True)

In [ ]:
X_s = explainer.X[:, explainer.explain_indices].clone() 
X_t = explainer.X_prime[:, explainer.explain_indices].clone()

In [ ]:
np.sqrt(explainer.swd.distance(X_s, X_t, delta)[0].item())

In [ ]:
explainer.swd.distance_interval(X_s, X_t, delta=delta, alpha=alpha, bootstrap=False)

In [ ]:
0.8*(1-0.04)**150

In [ ]:
explainer.optimize(U_1=0.1, U_2=0.25, l=0, r=1, max_iter=100, tau=1e1, kappa=0.02, bootstrap=True)

In [ ]:
# import os 
# import pickle 
# dump_data_path = './data/cardio/'
# with open(os.path.join(dump_data_path, "explainer_convergence.pkl"), "wb") as file:
#     pickle.dump(explainer, file)

In [ ]:
# import os 
# import pickle 
# dump_data_path = './data/cardio/'
# with open(os.path.join(dump_data_path, "explainer_convergence.pkl"), 'rb') as file:
#     explainer = pickle.load(file)

In [ ]:
s_qu = ""
s_u1 = ""

points_qu = list(map(lambda x: x.item(), explainer.swd_upper_list))
# points_list = list(map(lambda x: x.item(), explainer.swd_upper_list))[:iteration]
# points_list = list(map(lambda x: x.item() if type(x)==torch.Tensor else x, explainer.eta_list))
# points_list = explainer.interval_right_list

for i, v in enumerate(points_qu):
    s_qu += f"({i},{v})"
    s_u1 += f"({i},0.1)"
print("Qu, U1")
print(s_qu)
print(s_u1)

In [ ]:
s_qv = ""
s_u2 = ""

points_qv = list(map(lambda x: x.item(), explainer.wd_upper_list))

for i, v in enumerate(points_qv):
    s_qv += f"({i},{v})"
    s_u2 += f"({i},0.25)"
print("Qv, U2")
print(s_qv)
print(s_u2)

In [ ]:
s_eta = ""
s_eta_r = ""
s_eta_l = ""
eta_points = list(map(lambda x: x.item() if type(x)==torch.Tensor else x, explainer.eta_list))
eta_r_points = explainer.interval_left_list
eta_l_points = explainer.interval_right_list

for i, v in enumerate(eta_points):
    s_eta += f"({i},{v})"

for i, v in enumerate(eta_r_points):
    s_eta_r += f"({i},{v})"

for i, v in enumerate(eta_l_points):
    s_eta_l += f"({i},{v})"

print("eta, (l,r)")
print(s_eta)
print(s_eta_r)
print(s_eta_l)

In [ ]:
which_X = explainer.best_X

factual_X = df[df_X.columns].loc[indice].copy()
counterfactual_X = pd.DataFrame(
    which_X.detach().numpy() * std[df_X.columns].values + mean[df_X.columns].values,
    columns=df_X.columns,
)

dtype_dict = df.dtypes.apply(lambda x: x.name).to_dict()
for k, v in dtype_dict.items():
    if k in counterfactual_X.columns:
        if v[:3] == "int":
            counterfactual_X[k] = counterfactual_X[k].round().astype(v)
        else:
            counterfactual_X[k] = counterfactual_X[k].astype(v)

factual_y = pd.DataFrame(
    y.detach().numpy(), columns=[target_name], index=factual_X.index
)
counterfactual_y = pd.DataFrame(
    explainer.y.detach().numpy(), columns=[target_name], index=factual_X.index
)

# Recover the type of counterfactual_X
dtype_dict = df.dtypes.apply(lambda x: x.name).to_dict()
for k, v in dtype_dict.items():
    if k in counterfactual_X.columns:
        if v[:3] == "int":
            counterfactual_X[k] = counterfactual_X[k].round().astype(v)
        else:
            counterfactual_X[k] = counterfactual_X[k].astype(v)

counterfactual_X.index = factual_X.index
counterfactual_X[target_name] = counterfactual_y
factual_X[target_name] = factual_y

In [ ]:
pd.DataFrame({
    'factual_y': factual_y[target_name].values,
    'counterfactual_y': counterfactual_y[target_name].values,
})

In [ ]:
import os
import pickle 

dump_data_path = './data/cardio/'

factual_X.to_csv(os.path.join(dump_data_path, "factual.csv"), index=False)
counterfactual_X.to_csv(
    os.path.join(dump_data_path, "counterfactual.csv"), index=False
)
with open(os.path.join(dump_data_path, "explainer.pkl"), "wb") as file:
    pickle.dump(explainer, file)

In [ ]:
import matplotlib.pyplot as plt

fontsize = 25

# Enable LaTeX text rendering in Matplotlib
plt.rcParams.update({
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage{times}",  # Ensure you use the times package
    "font.family": "serif",
    "font.serif": ["Times", "Times New Roman"],  # This should use Times font
    "font.size": fontsize
})

matrix_nu = explainer.wd.nu.detach().numpy()

mu_avg = torch.zeros_like(explainer.swd.mu_list[0])
for mu in explainer.swd.mu_list:
    mu_avg += mu

total_sum = mu_avg.sum()

matrix_mu = mu_avg / total_sum

# Determine the global minimum and maximum values across both matrices
vmin = min(matrix_mu.min(), matrix_nu.min())
vmax = max(matrix_mu.max(), matrix_nu.max())

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(20, 8), facecolor='white')  # Set figure background to white

# Set the background of each axis to white
for ax in axs:
    ax.set_facecolor('white')

# First subplot for matrix_mu with 'Blues' color map
im_mu = axs[0].imshow(matrix_mu, cmap='Blues', vmin=vmin, vmax=vmax)
axs[0].set_title("Heatmap of $\mu$")

# Second subplot for matrix_nu with 'Blues' color map
im_nu = axs[1].imshow(matrix_nu, cmap='Blues', vmin=vmin, vmax=vmax)
axs[1].set_title("Heatmap of $\\nu$")

# Create a colorbar for the whole figure
fig.colorbar(im_mu, ax=axs, orientation='vertical')

plt.savefig('pictures/transportation_plan.eps', format='eps', bbox_inches='tight')


# # Display the plots
plt.show()
plt.close(fig)

In [ ]:
row_num  = 20

# Interleave rows
combined = pd.concat([factual_X.head(row_num), counterfactual_X.head(row_num)]).sort_index(kind='merge')

# Define formatters for specific columns
formatters = {
    "cardio": lambda x: f"{x:.4f}",
    "weight": lambda x: f"{x:.1f}"
}


# Convert to LaTeX
latex_code = combined.to_latex(index=False, formatters=formatters, 
                               caption="[\\textit{{German-Credit}}] Data points of factual and counterfactual distributions.", label="tab:german-credit")

print(latex_code)